# Working with text data

<img src="./images/chapter-2/big-picture-s1s1.png" width="720">

In [2]:
import re

In [3]:
with open("../data/the-verdict.txt", "r", encoding="utf-8") as f: 
    raw_text = f.read()

print("Total number of character:", len(raw_text))

Total number of character: 20479


### 1. Tokenizing Text

In [4]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text) 
preprocessed = [item.strip() for item in preprocessed if item.strip()] 
print(len(preprocessed))

4690


In [5]:
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


### 2. Converting tokens into token IDs

In [6]:
# sort all unique words and form a vocabulary
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [7]:
# map tokens to token ids
vocab = {token:integer for integer,token in enumerate(all_words)}

In [8]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


### 3. Implementing a simple text tokenizer

Let’s implement a complete tokenizer class in Python with an encode method that splits text into tokens and carries out the string-to-integer mapping to produce token IDs via the vocabulary. In addition, we’ll implement a decode method that carries out the reverse integer-to-string mapping to convert the token IDs back into text.

In [9]:
class SimpleTokenizerV1:
    def __init__(self, vocab: dict) -> None:
        self.str_to_int = vocab
        self.int_to_str = { v:k for k,v in vocab.items()}
    
    def encode(self, text:str) -> list[int]:
        # Processes input text into token IDs
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        # Converts token IDs back into text
        text = " ".join([self.int_to_str[i] for i in ids])
        # Removes spaces beforethespecified punctuation
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

#### sample test 1

In [10]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know,"
       Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [11]:
# from token ids to words
print(tokenizer.decode(ids))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


#### sample test 2

In [12]:
## error output code for learning purpose
# text = "Hello, do you like tea?"
# print(tokenizer.encode(text))

### 4. Adding special context tokens

In [13]:
all_tokens = sorted(list(set(preprocessed))) 
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}
print(len(vocab.items()))

1132


In [14]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [15]:
class SimpleTokenizerV2:
    def __init__(self, vocab: dict) -> None:
        self.str_to_int = vocab
        self.int_to_str = { v:k for k,v in vocab.items()}
    
    def encode(self, text:str) -> list[int]:
        # Processes input text into token IDs
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        # Replaces unknown words by <|unk|> tokens (changed)
        preprocessed = [
            item if item in self.str_to_int else '<|unk|>' for item in preprocessed
            ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        # Converts token IDs back into text
        text = " ".join([self.int_to_str[i] for i in ids])
        # Removes spaces before the specified punctuation (changed)
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [16]:
# we concatenate from two independent and unrelated sentences and test
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [17]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [18]:
# Let’s detokenize the text for a quick sanity check:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


In [19]:
# read page no 32 to understand other special tokens and their use

### 5. Byte pair encoding

In [20]:
import tiktoken

In [21]:
tokenizer = tiktoken.get_encoding("gpt2")

In [22]:
# encoding the texts
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace."
)
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [23]:
# decode the texts
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [24]:
# check
text = "Akwirw ier"
tok_ids = tokenizer.encode(text=text)
tok_ids

[33901, 86, 343, 86, 220, 959]

In [25]:
for item in tok_ids:
    decode_text = tokenizer.decode([item]) 
    print(f"tok_id - {item},\t decode_text - \'{decode_text}\'")

tok_id - 33901,	 decode_text - 'Ak'
tok_id - 86,	 decode_text - 'w'
tok_id - 343,	 decode_text - 'ir'
tok_id - 86,	 decode_text - 'w'
tok_id - 220,	 decode_text - ' '
tok_id - 959,	 decode_text - 'ier'


In [26]:
tokenizer.decode(tok_ids) 

'Akwirw ier'

### 6 Data sampling with a sliding window

In [27]:
# tokenize the whole “The Verdict” short story using the BPE tokenizer
with open("../data/the-verdict.txt", "r", encoding="utf-8") as f: 
    raw_text = f.read()
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [28]:
# Next, we remove the first 50 tokens from the dataset for demonstration purposes
enc_sample = enc_text[50:]

In [29]:
# One of the easiest and most intuitive ways to create the input–target pairs for the next- word prediction task is to create two variables, x and y, where x contains the input tokens and y contains the targets, which are the inputs shifted by 1:
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [30]:
# By processing the inputs along with the targets, which are the inputs shifted by one position, we can create the next-word prediction tasks
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [31]:
#  Let’s repeat the previous code but convert the token IDs into text:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [32]:
# There’s only one more task before we can turn the tokens into embeddings: imple- menting an efficient data loader that iterates over the input dataset and returns the inputs and targets as PyTorch tensors, which can be thought of as multidimensional arrays.

In [33]:
# While the figure shows the tokens in string format for illustration purposes, the code implementation will operate on token IDs directly since the encode method of the BPE tokenizer performs both tokenization and conversion into token IDs as a single step.

In [34]:
import torch
from torch.utils.data import Dataset, DataLoader

In [35]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride): 
            input_chunk = token_ids[i:i + max_length] 
            target_chunk = token_ids[i + 1: i + max_length + 1] 
            self.input_ids.append(torch.tensor(input_chunk)) 
            self.target_ids.append(torch.tensor(target_chunk))
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
    

In [36]:
# create data loader
def create_dataloader_v1(
        txt, batch_size=4, max_length=256, stride=128, shuffle=True,drop_last=True,num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
        )
    return dataloader

In [37]:
# Let’s test the dataloader with a batch size of 1 for an LLM with a context size of 4
with open("../data/the-verdict.txt", "r", encoding="utf-8") as f: 
    raw_text = f.read()

In [38]:
# create dataset
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=4, stride=2, shuffle=False
    )

In [39]:
# To understand the meaning of stride=1, let’s fetch another batch from this dataset:
data_iter = iter(dataloader)
for i in range(3):
    inputs, targets = next(data_iter)
    print(f"inputs:\n{inputs}")
    print(f"targets:\n{targets}")
    print("-"*20)

inputs:
tensor([[   40,   367,  2885,  1464],
        [ 2885,  1464,  1807,  3619],
        [ 1807,  3619,   402,   271],
        [  402,   271, 10899,  2138],
        [10899,  2138,   257,  7026],
        [  257,  7026, 15632,   438],
        [15632,   438,  2016,   257],
        [ 2016,   257,   922,  5891]])
targets:
tensor([[  367,  2885,  1464,  1807],
        [ 1464,  1807,  3619,   402],
        [ 3619,   402,   271, 10899],
        [  271, 10899,  2138,   257],
        [ 2138,   257,  7026, 15632],
        [ 7026, 15632,   438,  2016],
        [  438,  2016,   257,   922],
        [  257,   922,  5891,  1576]])
--------------------
inputs:
tensor([[ 922, 5891, 1576,  438],
        [1576,  438,  568,  340],
        [ 568,  340,  373,  645],
        [ 373,  645, 1049, 5975],
        [1049, 5975,  284,  502],
        [ 284,  502,  284, 3285],
        [ 284, 3285,  326,   11],
        [ 326,   11,  287,  262]])
targets:
tensor([[5891, 1576,  438,  568],
        [ 438,  568,  340,  

### 7. Creating token embeddings

In [40]:
# suppose we have 
vocab_size = 6
output_dim = 3

In [41]:
# Using the vocab_size and output_dim, we can instantiate an embedding layer in PyTorch, setting the random seed to 123 for reproducibility purposes:
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim) 
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [42]:
# The weight matrix of the embedding layer contains small, random values. These val- ues are optimized during LLM training as part of the LLM optimization itself. More- over, we can see that the weight matrix has six rows and three columns. There is one row for each of the six possible tokens in the vocabulary, and there is one column for each of the three embedding dimensions.

<span style="color:orange">**"In other words, the embedding layer is essentially a lookup operation that retrieves rows from the embedding layer’s weight matrix via a token ID."**</span>

<span style="color:green">*NOTE* For those who are familiar with one-hot encoding, the embedding layer approach described here is essentially just a more efficient way of implementing one-hot encoding followed by matrix multiplication in a fully connected layer, which is illustrated in the supplementary code on GitHub at https://mng.bz/ZEB5. Because the embedding layer is just a more efficient implementation equivalent to the one-hot encoding and matrix-multiplication approach, it can be seen as a neural network layer that can be optimized via backpropagation.</span>

- Also `torch.nn.Linear` and `torch.nn.Embedding` are same, we only use Embedding because of computation simplicity.

In [43]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [44]:
print(embedding_layer(torch.tensor([3,4])))

tensor([[-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315]], grad_fn=<EmbeddingBackward0>)


In [45]:
input_ids = torch.tensor([2, 3, 5, 1])
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


### 8. Encoding word positions

In principle, token embeddings are a suitable input for an LLM. However, a minor shortcoming of LLMs is that their self-attention mechanism doesn’t have a notion of position or order for the tokens within a sequence.

***Relative Positional Embeddings and Absolute Positional Embeddings***

**Absolute positional embeddings** are directly associated with specific positions in a sequence, For each posi- tion in the input sequence, a unique embedding is added to the token’s embedding to convey its exact location.